In [10]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [2]:
representations = np.load('/u/scr/ethanchi/relationOutputs/finnish/representations.npy')
morph = np.load('/u/scr/ethanchi/relationOutputs/finnish/morph.npy')
pos = np.load('/u/scr/ethanchi/relationOutputs/finnish/pos.npy')

In [3]:
CASES = ["Nom", "Acc", "Gen", "Ins", "Par", "Ess", "Tra", "Com", "Abe", "Ine", "Ill", "Ela", "Ade", "All", "Abl"]
NUM_CASES = len(CASES)

In [4]:
data = morph[pos=='NOUN']
reps = representations[pos=='NOUN']

print(data.shape)

labels = np.zeros([data.shape[0], NUM_CASES])

for idx, case in enumerate(CASES):
  is_case = np.vectorize(lambda x: x.startswith("Case=" + case))
  labels[:, idx] = is_case(data)

print(labels.shape)
  
labels = np.argmax(labels, axis=1)
print(labels.shape)
print(labels)

(5103,)
(5103, 15)
(5103,)
[ 2  0 12 ...  9 10  2]


In [5]:
label_tensor = torch.Tensor(labels)
data_tensor = torch.Tensor(reps)

indices = np.arange(label_tensor.shape[0])
np.random.shuffle(indices)

label_tensor = label_tensor[indices]
data_tensor = data_tensor[indices]

sep = int(0.9 * label_tensor.shape[0])

train_labels = label_tensor[:sep]
train_data = data_tensor[:sep]

test_labels = label_tensor[sep:]
test_data = data_tensor[sep:]

In [6]:
class CaseClassifier(nn.Module):
  def __init__(self, rep_size, out_size):
    super(CaseClassifier, self).__init__()
    self.linear = nn.Linear(rep_size, out_size)
  def forward(self, vectors):
    return nn.functional.log_softmax(self.linear(vectors), dim=1)
  
NUM_EPOCHS = 2000

model = CaseClassifier(representations.shape[-1], NUM_CASES)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = nn.CrossEntropyLoss()



for epoch in range(NUM_EPOCHS):
  optimizer.zero_grad()
  train_outputs = model(train_data)
  train_loss = loss(train_outputs, train_labels.long())
  train_loss.backward()
  optimizer.step()
  if epoch % 100 == 0:
    with torch.no_grad():
      test_outputs = model(test_data)
      test_loss = loss(test_outputs, test_labels.long())
      train_accuracy = len(np.where(torch.argmax(train_outputs, dim=1) == train_labels.long())[0]) / train_outputs.shape[0]
      test_accuracy = len(np.where(torch.argmax(test_outputs, dim=1) == test_labels.long())[0]) / test_outputs.shape[0]
      print("Epoch {}, train loss {:5.3f}, train accuracy {:2.3f}%, test loss {:5.3f}, test accuracy {:2.3f}%".format(epoch, train_loss, train_accuracy*100, test_loss, test_accuracy*100))



Epoch 0, train loss 2.900, train accuracy 2.983%, test loss 2.890, test accuracy 3.718%
Epoch 100, train loss 1.702, train accuracy 51.503%, test loss 1.752, test accuracy 48.141%
Epoch 200, train loss 1.230, train accuracy 72.300%, test loss 1.299, test accuracy 69.276%
Epoch 300, train loss 0.957, train accuracy 80.575%, test loss 1.030, test accuracy 76.517%
Epoch 400, train loss 0.780, train accuracy 85.540%, test loss 0.849, test accuracy 83.170%
Epoch 500, train loss 0.658, train accuracy 88.676%, test loss 0.722, test accuracy 86.497%
Epoch 600, train loss 0.569, train accuracy 90.418%, test loss 0.628, test accuracy 88.845%
Epoch 700, train loss 0.501, train accuracy 91.768%, test loss 0.557, test accuracy 89.628%
Epoch 800, train loss 0.448, train accuracy 92.726%, test loss 0.501, test accuracy 91.194%
Epoch 900, train loss 0.406, train accuracy 93.423%, test loss 0.457, test accuracy 91.389%
Epoch 1000, train loss 0.370, train accuracy 94.077%, test loss 0.420, test accuracy

In [7]:
en_representations = np.load('/u/scr/ethanchi/relationOutputs/english/representations.npy')
en_pos = np.load('/u/scr/ethanchi/relationOutputs/english/pos.npy')
en_extra = np.load('/u/scr/ethanchi/relationOutputs/english/data.npy')

en_extra = en_extra[en_pos=='NOUN']
en_representations = en_representations[en_pos=='NOUN']

print(en_representations.shape)

(4195, 768)


In [8]:
en_inputs = torch.Tensor(en_representations)
with torch.no_grad():
  en_outputs = model(en_inputs)
  en_best_labels = torch.argmax(en_outputs, dim=1)
  
for case, num in zip(CASES, torch.bincount(en_best_labels.int())):
  print(f"{case}\t{num}")
  
  


Nom	3151
Acc	0
Gen	301
Ins	2
Par	545
Ess	34
Tra	2
Com	0
Abe	0
Ine	60
Ill	15
Ela	6
Ade	63
All	11
Abl	5


In [11]:
for idx, (case, num) in enumerate(zip(CASES, torch.bincount(en_best_labels.int()))):
  print(f"{case}\t{num}")
  for (sentence, sent_idx, _) in en_extra[en_best_labels.numpy()==idx][:10]:
      sent_idx = int(sent_idx)
      words = sentence.split(' ')
      words[sent_idx] = "**" + words[sent_idx] + "**"
      printmd(" ".join(words))

Nom	3151


From the AP comes this **story** :

President Bush on Tuesday nominated two **individuals** to replace retiring jurists on federal courts in the Washington area .

President Bush on Tuesday nominated two individuals to replace retiring **jurists** on federal courts in the Washington area .

President Bush on Tuesday nominated two individuals to replace retiring jurists on federal courts in the Washington **area** .

...

It is true that Google has been in acquisition **mode** .

Raw data is on the fisrt tab of the file to the **right** .

Normally this author is semi-objective ( what blogger is ) but on the Seattlepi.com ( see source link ) Palestinian Leader Mahmoud Abbas has demanded that Israel leave all the land that it occupies before the 1967 border ( which includes East Jerusalem by the **way** ) .

" We 're not happy with Hamas 's position at this **stage** and in light of the Israeli escalation against the Palestinians , " commented Khader Habib , a senior Islamic Jihad official .

Miramar California is a **bit** north of San Diego .

In [12]:
en_non_nom_outputs = en_outputs[:, 1:]
en_rev_best_labels = torch.argmax(en_non_nom_outputs, dim=1)
  
for case, num in zip(CASES[1:], torch.bincount(en_rev_best_labels.int())):
  print(f"{case}\t{num}")
  
  

Acc	0
Gen	1524
Ins	15
Par	2009
Ess	68
Tra	6
Com	0
Abe	0
Ine	151
Ill	122
Ela	23
Ade	227
All	36
Abl	14


In [13]:
for idx, (case, num) in enumerate(zip(CASES[1:], torch.bincount(en_rev_best_labels.int()))):
  print(f"{case}\t{num}")
  print(en_extra[en_rev_best_labels.numpy()==idx][:10])

Acc	0
[]
Gen	1524
[['The sheikh in wheel - chair has been attacked with a F - 16 - launched bomb .'
  '11' 'F']
 ["Today 's incident proves that Sharon has lost his patience and his hope in peace ."
  '2' 'incident']
 ["Today 's incident proves that Sharon has lost his patience and his hope in peace ."
  '9' 'patience']
 ['In the eastern city of Baqubah , guerrillas detonated a car bomb outside a police station , killing several people .'
  '10' 'car']
 ['In the eastern city of Baqubah , guerrillas detonated a car bomb outside a police station , killing several people .'
  '14' 'police']
 ['( I hope that the US army got an enormous amount of information from her relatives , because otherwise this move was a bad , bad tradeoff ) .'
  '6' 'army']
 ['( I hope that the US army got an enormous amount of information from her relatives , because otherwise this move was a bad , bad tradeoff ) .'
  '26' 'tradeoff']
 ['The US troops fired into the hostile crowd , killing 4 .' '2' 'troops']
 ['Th

In [14]:
def printmd(string):
    display(Markdown(string))
for idx, (case, num) in enumerate(zip(CASES[1:], torch.bincount(en_rev_best_labels.int()))):
  print(f"{case}\t{num}")
  for (sentence, sent_idx, _) in en_extra[en_rev_best_labels.numpy()==idx][:10]:
    sent_idx = int(sent_idx)
    words = sentence.split(' ')
    words[sent_idx] = "**" + words[sent_idx] + "**"
    printmd(" ".join(words))

Acc	0
Gen	1524


The sheikh in wheel - chair has been attacked with a **F** - 16 - launched bomb .

Today 's **incident** proves that Sharon has lost his patience and his hope in peace .

Today 's incident proves that Sharon has lost his **patience** and his hope in peace .

In the eastern city of Baqubah , guerrillas detonated a **car** bomb outside a police station , killing several people .

...

Vangie McGilloway Constellation Power Source , Inc. ( " CPS " ) 111 Market Place **Ste** 500 Baltimore , MD 21202 Phone 410-468-3798 Fax 410-468-3499 Email vangie.mcgilloway@powersrc.com

I hope you have a good flight back to **home** .

Sungjoo Lee Stochastic Processes and Simulation in Finance Georgia Institute of Technology School of Industrial and Systems Engineering E-mail address : goldconn@isye.gatech.edu gte114t@prism.gatech.edu **Home** : 404-449-1026 Office : Coming Soon !

The storm threatened oil installations in the Gulf of Mexico where about one - **quarter** of US oil operations are based .

Normally this author is semi-objective ( what blogger is ) but on the Seattlepi.com ( see source link ) Palestinian Leader Mahmoud Abbas has demanded that Israel leave all the land that it occupies before the 1967 border ( which includes East Jerusalem by the **way** ) .

In [15]:
for idx, (case, num) in enumerate(zip(CASES, torch.bincount(en_best_labels.int()))):
  print(f"{case}\t{num}")
#   if num == 0: continue
  for (sentence, sent_idx, _), score in zip(en_extra[np.argsort(-en_outputs[:, idx])][:10], en_outputs[np.argsort(-en_outputs[:, idx]), idx][:10]):
      sent_idx = int(sent_idx)
      words = sentence.split(' ')
      words[sent_idx] = "**" + words[sent_idx] + "**"
      words = " ".join(words)
      words = f"`({np.exp(score):5.3f})` " + words
      printmd(words)

Nom	3151


`(0.999)` The chicken cordon - **blu** was tasty and came in a huge portion size for the money .

`(0.998)` My old **bed** got tossed in a dumpster .

`(0.998)` The **sheikh** in wheel - chair has been attacked with a F - 16 - launched bomb .

`(0.998)` The **owner** Karla is welcoming and fun .

...

`(0.243)` In addition , I received feedback from our Gas Desk that the access to the Gas **segment** of Enron On - Line was cut off to CPS --- do you know who would handle this at Enron that we can speak to ?

`(0.241)` The Board of Muslim clerics in Fallujah , 50 **km** west of Baghdad , " condemned the assassination and promised immediate revenge against the coalition soldiers in Iraq . "

`(0.206)` I have no inside information , but I have been following links **today** that strongly indicate that Google is damn serious about securing permanent control of the leading edge browser technology in Firefox .

`(0.204)` Choose this hotel over the Hilton ( which is on the next **block** ) .

`(0.199)` It is true that Google has been in acquisition **mode** .

In [16]:
print(representations.shape)

(18290, 768)
